## Package Imports and Variables

In [ ]:
import requests

keycloak_root = "http://localhost:8642"
keycloak_admin = "admin"
keycloak_admin_password = "admin"

## Get Access Token

In [ ]:
resp = requests.post(
    f"{keycloak_root}/realms/master/protocol/openid-connect/token",
    data={
        "client_id": "admin-cli",
        "username": keycloak_admin,
        "password": keycloak_admin_password,
        "grant_type": "password"
    }
)
resp.raise_for_status()
data = resp.json()
access_token = data["access_token"]
print(f"{access_token[:20]}...{access_token[-20:]}")
print(f"Expires in {data['expires_in']}s")

# Predefine authorization headers for later use.
auth_headers = {
    "Authorization": f"Bearer {access_token}",
}

## Get Realms

In [ ]:
resp = requests.get(
    f"{keycloak_root}/admin/realms",
    headers=auth_headers,
)
resp.raise_for_status()
[r["realm"] for r in resp.json()]

## Create Realm

In [ ]:
# Name of our fancy realm.
realm = "myrealm"

In [ ]:
resp = requests.post(
    f"{keycloak_root}/admin/realms",
    headers=auth_headers,
    json={
        "realm": realm,
        "enabled": True
    }
)
resp.raise_for_status()

## List Realms

In [ ]:
resp = requests.get(
    f"{keycloak_root}/admin/realms",
    headers=auth_headers,
)
resp.raise_for_status()
[r["realm"] for r in resp.json()]

## Create Client

In [ ]:
client_id = "fancy-client-1"

client_settings = {
    "protocol": "openid-connect",
    "clientId": client_id,
    "enabled": True,
    # Public: no client secret. Non-public: "confidential" client with secret.
    "publicClient": True,
    # Authorization Code Flow
    "standardFlowEnabled": True,
    # Service accounts: Client Credentials Grant
    "serviceAccountsEnabled": False,
    # Direct Access: Resource Owner Password Credentials Grant
    "directAccessGrantsEnabled": True,
    "attributes": {
        # Device authorization grant
        "oauth2.device.authorization.grant.enabled": True,
    }
}

resp = requests.post(
    f"{keycloak_root}/admin/realms/{realm}/clients",
    json=client_settings,
    headers=auth_headers,
)
resp.raise_for_status()
location = resp.headers["Location"]
print(location)

## Get Client Details

In [ ]:
requests.get(
    location,
    headers=auth_headers,
).json()

## Create User

In [ ]:
username = "bob"
password = "bob"

user_settings = {
    "username": username,
    "enabled": True,
    "credentials": [{
        "type": "password",
        "value": password,
        "temporary": False,
    }]
}


resp = requests.post(
    f"{keycloak_root}/admin/realms/{realm}/users",
    json=user_settings,
    headers=auth_headers,
)
resp.raise_for_status()
location = resp.headers["Location"]
print(location)

## Get User Details

In [ ]:
requests.get(
    location,
    headers=auth_headers,
).json()

## Get Token Endpoint of the Realm

In [ ]:
token_endpoint = requests.get(
    f"{keycloak_root}/realms/{realm}/.well-known/openid-configuration"
).json()["token_endpoint"]
print(token_endpoint)

## Do the token request

In [ ]:
resp = requests.post(
    token_endpoint,
    data={
        "client_id": client_id,
        "username": username,
        "password": password,
        "grant_type": "password",
    }
)
resp.raise_for_status()
resp.json()